In [1]:
%load_ext tensorboard

import tensorflow as tf
from tensorboard.plugins.hparams import api as hp

In [2]:
!rm -rf ./logs/hp_tuning

In [3]:
fashion_mnist = tf.keras.datasets.fashion_mnist

(x_train, y_train),(x_test, y_test) = fashion_mnist.load_data()

In [4]:
x_train, x_test = x_train / 255.0, x_test / 255.0

In [5]:
HP_UNITS = hp.HParam('units', hp.Discrete([16, 32, 64]))
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.1, 0.2))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adamax', 'sgd']))

METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('logs/hp_tuning').as_default():
    hp.hparams_config(
        hparams=[HP_UNITS, HP_DROPOUT, HP_OPTIMIZER],
        metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
    )

In [6]:
def step(hparams):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(hparams[HP_UNITS], activation=tf.nn.relu))
    model.add(tf.keras.layers.Dropout(hparams[HP_DROPOUT]))
    model.add(tf.keras.layers.Dense(10, activation=tf.nn.softmax))

    model.compile(
        optimizer=hparams[HP_OPTIMIZER],
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'],
    )

    model.fit(
        x_train,
        y_train, 
        epochs=1
    )
    
    loss, accuracy = model.evaluate(x_test, y_test)
    return accuracy

In [7]:
def run(run_dir, hparams):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)
        accuracy = step(hparams)
        tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

In [8]:
session = 0

%tensorboard --logdir logs/hp_tuning


for units in HP_UNITS.domain.values:
    for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
        for optimizer in HP_OPTIMIZER.domain.values:
            hparams = {
                HP_UNITS: units,
                HP_DROPOUT: dropout_rate,
                HP_OPTIMIZER: optimizer,
            }
            
            run_name = "run-%d" % session
            print('Starting Trial: %s' % run_name)
            print({h.name: hparams[h] for h in hparams})
            run('logs/hp_tuning/' + run_name, hparams)
            session += 1

Starting Trial: run-0
{'units': 16, 'dropout': 0.1, 'optimizer': 'adamax'}
Train on 60000 samples
10000/10000 [==============================] - 0s 38us/sample - loss: 0.5871 - accuracy: 0.7944
Starting Trial: run-1
{'units': 16, 'dropout': 0.1, 'optimizer': 'sgd'}
Train on 60000 samples
10000/10000 [==============================] - 0s 27us/sample - loss: 0.6668 - accuracy: 0.7674
Starting Trial: run-2
{'units': 16, 'dropout': 0.2, 'optimizer': 'adamax'}
Train on 60000 samples
10000/10000 [==============================] - 0s 27us/sample - loss: 0.6262 - accuracy: 0.7895
Starting Trial: run-3
{'units': 16, 'dropout': 0.2, 'optimizer': 'sgd'}
Train on 60000 samples
10000/10000 [==============================] - 0s 28us/sample - loss: 0.7716 - accuracy: 0.7387
Starting Trial: run-4
{'units': 32, 'dropout': 0.1, 'optimizer': 'adamax'}
Train on 60000 samples
10000/10000 [==============================] - 0s 27us/sample - loss: 0.5328 - accuracy: 0.8174
Starting Trial: run-5
{'units': 32, 

### Using Keras Callback

In [8]:
logdir = './logs/hp_tuning'

In [9]:
for units in HP_UNITS.domain.values:
    for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
        for optimizer in HP_OPTIMIZER.domain.values:
            hparams = {
                HP_UNITS: units,
                HP_DROPOUT: dropout_rate,
                HP_OPTIMIZER: optimizer,
            }
            
            model = tf.keras.models.Sequential()
            model.add(tf.keras.layers.Flatten())
            model.add(tf.keras.layers.Dense(hparams[HP_UNITS], activation=tf.nn.relu))
            model.add(tf.keras.layers.Dropout(hparams[HP_DROPOUT]))
            model.add(tf.keras.layers.Dense(10, activation=tf.nn.softmax))
            
            model.compile(
                optimizer=hparams[HP_OPTIMIZER],
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'],
            )
            
            print({h.name: hparams[h] for h in hparams})
            model.fit(
                x_train,
                y_train, 
                epochs=1,
                callbacks=[
                    tf.keras.callbacks.TensorBoard(logdir),
                    hp.KerasCallback(logdir, hparams),
                ],
            )


{'units': 16, 'dropout': 0.1, 'optimizer': 'adamax'}
Train on 60000 samples
60000/60000 [==============================] - 3s 42us/sample - loss: 0.9097 - accuracy: 0.6992
{'units': 16, 'dropout': 0.1, 'optimizer': 'sgd'}
Train on 60000 samples
60000/60000 [==============================] - 2s 40us/sample - loss: 1.0683 - accuracy: 0.6396
{'units': 16, 'dropout': 0.2, 'optimizer': 'adamax'}
Train on 60000 samples
60000/60000 [==============================] - 3s 42us/sample - loss: 1.0010 - accuracy: 0.6632
{'units': 16, 'dropout': 0.2, 'optimizer': 'sgd'}
Train on 60000 samples
60000/60000 [==============================] - 2s 40us/sample - loss: 1.0641 - accuracy: 0.6241
{'units': 32, 'dropout': 0.1, 'optimizer': 'adamax'}
Train on 60000 samples
60000/60000 [==============================] - 3s 43us/sample - loss: 0.7382 - accuracy: 0.7504
{'units': 32, 'dropout': 0.1, 'optimizer': 'sgd'}
Train on 60000 samples
60000/60000 [==============================] - 2s 41us/sample - loss: 0.9